In [1]:
import pandas as pd
import numpy as np
import tensorflow

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

seed = 2000
np.random.seed(seed)
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense, Embedding
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing import sequence

from gensim.models import Word2Vec
from gensim.parsing.porter import PorterStemmer

import nltk
from nltk.tokenize import TweetTokenizer

import warnings
warnings.filterwarnings("ignore")

2023-01-05 01:17:03.069690: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_sub1_df = pd.read_csv('/Users/nitanshjain/Documents/Miscellaneous/SemEval/semeval2023task3/preprocessed_data/subtask1/train_subtask_1.csv')
print(train_sub1_df.shape)
train_sub1_df.head()
print(train_sub1_df.genre.value_counts())

(433, 10)
opinion      382
reporting     41
satire        10
Name: genre, dtype: int64


In [3]:
le = LabelEncoder()
train_sub1_df['genre'] = le.fit_transform(train_sub1_df['genre'])
print(train_sub1_df.genre.value_counts())
train_sub1_df.head()

0    382
1     41
2     10
Name: genre, dtype: int64


,id,genre,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
0,833042063,2,Chelsea Handler Admits She’s ‘Very Sexually At...,Far-left comedienne Chelsea Handler has admitt...,chelsea handler admit she s very sexually attr...,"[(Chelsea, 'NNP'), (Handler, 'NNP'), (admit, '...","[(Chelsea Handler, 'PERSON', 380), (Robert Mue...",far leave comedienne chelsea handler have admi...,"[(far, 'RB'), (leave, 'VB'), (comedienne, 'NNP...","[(Chelsea Handler, 'PERSON', 380), (FBI Specia..."
1,832959523,2,How Theresa May Botched\n,Those were the times…\nThe Times page 1 is of ...,how theresa may botch,"[(how, 'WRB'), (Theresa, 'NNP'), (May, 'NNP'),...","[(Theresa, 'GPE', 384), (May, 'DATE', 391)]",those be the time the times page 1 be of janua...,"[(those, 'DT'), (be, 'VBP'), (the, 'DT'), (tim...","[(Times, 'ORG', 383), (1, 'CARDINAL', 397), (J..."
2,833039623,2,Robert Mueller III Rests His Case—Dems NEVER W...,Carload of crazies headed to the White House w...,robert mueller iii rest his case dems never will,"[(Robert, 'NNP'), (Mueller, 'NNP'), (III, 'NNP...","[(Robert Mueller III, 'PERSON', 380), (Dems, '...",carload of crazy head to the white house want ...,"[(Carload, 'NNP'), (of, 'IN'), (crazy, 'JJ'), ...","[(the White House, 'ORG', 383), (Barack Obama,..."
3,833032367,2,Robert Mueller Not Recommending Any More Indic...,"But of course, this makes no difference to the...",robert mueller not recommend any more indictment,"[(Robert, 'NNP'), (Mueller, 'NNP'), (not, 'RB'...","[(Robert Mueller, 'PERSON', 380)]",but of course this make no difference to the p...,"[(but, 'CC'), (of, 'IN'), (course, 'NN'), (thi...","[(the New York Times, 'ORG', 383), (late Frida..."
4,814777937,2,The Far Right Is Trying to Co-opt the Yellow V...,"This weekend in Manchester, England, Yellow Ve...",the far right be try to co opt the yellow vests,"[(the, 'DT'), (Far, 'NNP'), (right, 'NN'), (be...",[],this weekend in manchester england yellow vest...,"[(this, 'DT'), (weekend, 'NN'), (in, 'IN'), (M...","[(this weekend, 'DATE', 391), (Manchester, 'GP..."


In [4]:
def creating_tokens(df):
    articles = df.preprocessed_articles
    headlines = df.preprocessed_headlines
    texts = articles + headlines
    
    tokens = list()
    tokenizer = TweetTokenizer()
    
    for text in texts:
        tokens.append(tokenizer.tokenize(text))
    
    df['tokens_text'] = tokens
    
    porter_stemmer = PorterStemmer()
    # Get the stemmed_tokens
    df['stemmed_tokens_text'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in df['tokens_text']]
    
    return df

In [5]:
train_sub1_df = creating_tokens(train_sub1_df)
x_tokens = train_sub1_df.stemmed_tokens_text
x_tokens = x_tokens.to_frame().reset_index()

y = train_sub1_df.genre

OUTPUT_FOLDER = '/Users/nitanshjain/Documents/Miscellaneous/SemEval/semeval2023task3/codefiles/subtask1/word2vec/cbow/text/'

tokens = pd.Series(train_sub1_df.stemmed_tokens_text).values
# print(tokens)
word2vec_model_file = OUTPUT_FOLDER + 'word2vec_subtask1_' + str(200) + '.model'

w2v_model = Word2Vec(tokens, min_count=1, vector_size=200, window=5, workers=4, sg=2)
w2v_model.train(tokens, epochs=10, total_examples=len(tokens))
w2v_model.save(word2vec_model_file)

In [6]:
def create_file(create_file, model_file, x):
    sg_w2v_model = Word2Vec.load(model_file)
    
    with open(create_file, 'w+') as word2vec_file:
        for index, row in x.iterrows():
            model_vector = (np.mean([sg_w2v_model.wv[token] for token in row['stemmed_tokens_text']], axis=0)).tolist()
            if index == 0:
                header = ",".join(str(ele) for ele in range(200))
                word2vec_file.write(header)
                word2vec_file.write("\n")
            
            if type(model_vector) is list:
                line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
            else:
                line1 = ",".join([str(0) for i in range(200)])
            word2vec_file.write(line1)
            word2vec_file.write('\n')
    
    df = pd.read_csv(create_file)
    return df
        

In [7]:
word2vec_train_filename = OUTPUT_FOLDER + 'word2vec_subtask1_train_' + str(200) + '.csv'
word2vec_train_df = create_file(word2vec_train_filename, word2vec_model_file, x_tokens)
print(word2vec_train_df.shape)
word2vec_train_df.head()

(433, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.057023,-0.046974,-0.119872,0.074033,0.093780,-0.163542,0.054604,0.352836,0.009015,0.010837,...,0.211090,0.031598,-0.157998,0.000963,-0.020315,0.083703,-0.062942,-0.089271,-0.019336,-0.022333
1,0.073339,-0.025183,-0.065871,0.125752,0.101605,-0.101224,0.067087,0.359594,0.035522,0.031038,...,0.147971,0.006097,-0.123449,-0.022419,-0.049047,0.088016,-0.073154,-0.062970,0.001119,-0.094848
2,0.066100,-0.016196,-0.102681,0.112423,0.107274,-0.121176,0.055603,0.321244,0.019246,0.030151,...,0.130723,0.035938,-0.119866,-0.009457,-0.027275,0.080994,-0.098678,-0.070525,-0.040590,-0.040808
3,0.117753,-0.041735,-0.092932,0.142670,0.078955,-0.188116,-0.010440,0.377951,0.029416,-0.020622,...,0.145707,-0.009353,-0.132705,-0.005330,-0.069417,0.072274,-0.082185,-0.076866,-0.029585,-0.020619
4,0.106022,0.006376,-0.010921,0.079795,0.062115,-0.141638,0.036544,0.301768,-0.060693,0.068646,...,0.099761,0.048389,-0.096497,-0.101092,0.004428,0.025751,-0.113758,-0.060745,-0.052044,-0.072730


In [8]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()

x_scaled = pd.DataFrame(mm.fit_transform(word2vec_train_df))
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=seed)
x_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.422275,0.443241,0.143030,0.265252,0.583837,0.552153,0.654039,0.609157,0.440488,0.432760,...,0.939529,0.585157,0.191353,0.761807,0.523067,0.671572,0.725099,0.438314,0.516057,0.635049
1,0.505764,0.577355,0.345767,0.578049,0.625683,0.791262,0.706424,0.644080,0.562323,0.513028,...,0.579259,0.431321,0.363042,0.657239,0.411289,0.689004,0.663997,0.632746,0.653904,0.235455
2,0.468722,0.632662,0.207572,0.497436,0.655996,0.714711,0.658230,0.445878,0.487514,0.509504,...,0.480809,0.611343,0.380845,0.715209,0.495990,0.660623,0.511274,0.576900,0.372825,0.533243
3,0.733031,0.475485,0.244171,0.680367,0.504566,0.457867,0.381070,0.738956,0.534257,0.307756,...,0.566334,0.338118,0.317042,0.733664,0.332038,0.625381,0.609959,0.530017,0.446988,0.644494
4,0.673003,0.771579,0.552064,0.300101,0.414522,0.636197,0.578244,0.345223,0.120088,0.662468,...,0.304081,0.686455,0.496977,0.305403,0.619326,0.437362,0.421042,0.649195,0.295636,0.357337


# KNN

In [9]:
from sklearn.neighbors import KNeighborsClassifier
knn_params = {
    'n_neighbors':range(1,15),
    'weights':['uniform', 'distance']
}

knn = KNeighborsClassifier()
clf = GridSearchCV(knn, knn_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8899224806201552
Best Parameters {'n_neighbors': 7, 'weights': 'uniform'}


# Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression
lr_params = {
    'C':np.logspace(-2,2,7), 
    'penalty':['l1', 'l2', 'elasticnet', None]
}

lr = LogisticRegression()
clf = GridSearchCV(lr, lr_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8853241719520792
Best Parameters {'C': 0.21544346900318834, 'penalty': 'l2'}


# Decison Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier

dt_params = {
    'max_depth':range(2,15),
    'criterion':['gini', 'entropy', 'log_loss'],
    'splitter':['best', 'random']
}

dt = DecisionTreeClassifier()
clf = GridSearchCV(dt, dt_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8883544749823821
Best Parameters {'criterion': 'entropy', 'max_depth': 2, 'splitter': 'best'}


# Random Forest Classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier

rfc_params = {
    'max_depth':range(3,15),
    'criterion':['gini', 'entropy', 'log_loss'],
}

rfc = RandomForestClassifier()
clf = GridSearchCV(rfc, rfc_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.890697674418605
Best Parameters {'criterion': 'gini', 'max_depth': 4}


# Multinomial Naive Bayes

In [13]:
%%time
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(x_scaled, y)

print('Train Accuracy Score', mnb.score(x_scaled, y))

Train Accuracy Score 0.8845265588914549
CPU times: user 10.6 ms, sys: 8.98 ms, total: 19.6 ms
Wall time: 20.2 ms


# SVM

In [14]:
from sklearn.svm import SVC

svc_params = {
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
    'degree':range(2,5)
}

svc = SVC()
clf = GridSearchCV(svc, svc_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8853241719520791
Best Parameters {'degree': 2, 'kernel': 'linear'}


# Ada-Boosting

In [15]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(x_scaled, y)

print('Train Accuracy Score', ada.score(x_scaled, y))

Train Accuracy Score 0.7413394919168591


# LSTM

In [ ]:
x_scaled = x_scaled[:,:,None]

In [ ]:
from keras import Sequential
from keras.layers import LSTM

model_lstm = Sequential()

model_lstm.add(LSTM(64, input_shape = x_scaled.shape[1:], return_sequences = True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.8))

model_lstm.add(LSTM(32, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(16, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(8, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(1, return_sequences=True, name='output'))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(Dense(64, activation = 'relu'))
model_lstm.add(Flatten())
model_lstm.add(Dense(1, activation='softmax'))

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

model_lstm.summary()

2022-12-27 14:13:43.797353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 11720, 64)         16896     
                                                                 
 dropout (Dropout)           (None, 11720, 64)         0         
                                                                 
 lstm_1 (LSTM)               (None, 11720, 32)         12416     
                                                                 
 dropout_1 (Dropout)         (None, 11720, 32)         0         
                                                                 
 lstm_2 (LSTM)               (None, 11720, 16)         3136      
                                                                 
 dropout_2 (Dropout)         (None, 11720, 16)         0         
                                                                 
 lstm_3 (LSTM)               (None, 11720, 8)          8

In [ ]:
batch_size = 32

model_lstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)

Epoch 1/5
4/7 [================>.............] - ETA: 2:36 - loss: 0.6169 - accuracy: 0.0859

# CNN LSTM

In [ ]:
from keras import Sequential
from keras.layers import LSTM

model_clstm  =  Sequential()

model_clstm.add(Conv1D(32, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(Conv1D(16, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(Conv1D(8, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(LSTM(32, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(LSTM(16, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(LSTM(1, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(Dense(64, activation = 'relu'))
model_clstm.add(Flatten())
model_clstm.add(Dense(1, activation = 'softmax'))

model_clstm.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model_clstm.summary()

In [ ]:
batch_size = 32

model_clstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)

# Bi-LSTM

In [ ]:
from keras import Sequential
from keras.layers import LSTM, Embedding, Bidirectional


model_bilstm  =  Sequential()

model_bilstm.add(Bidirectional(LSTM(32, input_shape = x_scaled.shape[1:], return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Bidirectional(LSTM(16, return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Bidirectional(LSTM(1, return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Dense(64, activation = 'relu'))
model_bilstm.add(Flatten())
model_bilstm.add(Dense(1, activation = 'softmax'))

model_bilstm.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model_bilstm.summary()

In [ ]:
batch_size = 32

model_bilstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)